# Bitcoin Price Prediction with TimeGPT

This notebook provides an interactive exploration of Bitcoin price prediction using TimeGPT.

## Setup

First, install the required packages and import libraries.

In [ ]:
# Install required packages
!pip install nixtla pandas matplotlib seaborn scikit-learn python-dotenv -q

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from nixtla import NixtlaClient
from dotenv import load_dotenv

# Add src to path
sys.path.insert(0, 'src')

from data_utils import load_bitcoin_data, add_derived_features
from backtesting import BitcoinBacktester
from analysis import BitcoinAnalyzer

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

# Load environment variables
load_dotenv()

print("✓ Libraries imported successfully")

## Initialize TimeGPT Client

Set your API key (get one at https://dashboard.nixtla.io)

In [ ]:
# Set your API key here or use environment variable
api_key = os.environ.get('NIXTLA_API_KEY', 'your-api-key-here')

client = NixtlaClient(api_key=api_key)

print("✓ TimeGPT client initialized")

## Load Bitcoin Price Data

In [ ]:
# Load Bitcoin price history
df = load_bitcoin_data()

print(f"Loaded {len(df)} days of Bitcoin price history")
print(f"Date range: {df['ds'].min()} to {df['ds'].max()}")
print(f"Price range: ${df['y'].min():.2f} to ${df['y'].max():.2f}")

df.head()

## Visualize Historical Prices

In [ ]:
# Plot full history
plt.figure(figsize=(14, 6))
plt.plot(df['ds'], df['y'], linewidth=1.5)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Price (USD)', fontsize=12)
plt.title('Bitcoin Price History', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Quick Forecast Example

Let's make a simple forecast for the next 30 days using the most recent data.

In [ ]:
# Use last 365 days for training
train_df = df.tail(365).copy()

# Forecast next 30 days with confidence intervals
forecast_df = client.forecast(
    df=train_df,
    h=30,
    level=[80, 95]
)

forecast_df.head()

In [ ]:
# Plot the forecast
client.plot(train_df, forecast_df, level=[80, 95])

## Backtesting Analysis

Now let's perform a backtesting analysis to evaluate prediction accuracy.

In [ ]:
# Initialize backtester
backtester = BitcoinBacktester(client, df)
analyzer = BitcoinAnalyzer()

print("✓ Backtester initialized")

### Test Recent Period (Last 90 days)

In [ ]:
# Define test period
max_date = df['ds'].max()
start_date = (max_date - timedelta(days=90)).strftime('%Y-%m-%d')
end_date = max_date.strftime('%Y-%m-%d')

print(f"Backtesting period: {start_date} to {end_date}")

# Run backtest
results = backtester.backtest_period(
    start_date=start_date,
    end_date=end_date,
    horizon=1,
    window_size=180,
    step_size=7,
    level=[80, 95]
)

print(f"\nGenerated {len(results)} predictions")
results.head()

### Calculate Performance Metrics

In [ ]:
# Calculate metrics
metrics = backtester.calculate_metrics(results)
directional_metrics = backtester.evaluate_directional_accuracy(results)

print("Performance Metrics:")
print(f"  MAE: ${metrics['mae']:.2f}")
print(f"  RMSE: ${metrics['rmse']:.2f}")
print(f"  MAPE: {metrics['mape']:.2f}%")
print(f"  80% Coverage: {metrics.get('coverage_80', 0):.2%}")
print(f"  95% Coverage: {metrics.get('coverage_95', 0):.2%}")
print("\nDirectional Accuracy:")
print(f"  Accuracy: {directional_metrics['accuracy']:.2%}")
print(f"  Precision: {directional_metrics['precision']:.2%}")
print(f"  Recall: {directional_metrics['recall']:.2%}")
print(f"  F1 Score: {directional_metrics['f1_score']:.2%}")

### Visualize Results

In [ ]:
# Plot forecast vs actual
analyzer.plot_forecast_vs_actual(
    results,
    title="Bitcoin Price: Forecast vs Actual (Recent Period)"
)
plt.show()

In [ ]:
# Plot error distribution
analyzer.plot_error_distribution(
    results,
    title="Forecast Error Distribution (Recent Period)"
)
plt.show()

In [ ]:
# Plot performance over time
analyzer.plot_performance_over_time(
    results,
    title="Forecast Performance Over Time (Recent Period)"
)
plt.show()

## Test Different Horizons

Let's compare 1-day, 7-day, and 30-day forecasts.

In [ ]:
horizons = [1, 7, 30]
horizon_results = {}

for h in horizons:
    print(f"\nTesting {h}-day forecast...")
    
    results_h = backtester.backtest_period(
        start_date=start_date,
        end_date=end_date,
        horizon=h,
        window_size=365,
        step_size=h,
        level=[80, 95]
    )
    
    if len(results_h) > 0:
        metrics_h = backtester.calculate_metrics(results_h)
        horizon_results[f'{h}-day'] = metrics_h
        print(f"  MAPE: {metrics_h['mape']:.2f}%")
        print(f"  Predictions: {len(results_h)}")

In [ ]:
# Compare horizons
comparison_df = pd.DataFrame(horizon_results).T
comparison_df[['mae', 'rmse', 'mape']].plot(kind='bar', figsize=(12, 6))
plt.title('Performance by Forecast Horizon', fontsize=14, fontweight='bold')
plt.xlabel('Forecast Horizon', fontsize=12)
plt.ylabel('Error', fontsize=12)
plt.legend(['MAE (USD)', 'RMSE (USD)', 'MAPE (%)'])
plt.xticks(rotation=0)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Explore Specific Predictions

Let's look at individual predictions in detail.

In [ ]:
# Show predictions with largest errors
top_errors = results.nlargest(5, 'abs_error')[[
    'forecast_date', 'actual', 'predicted', 'error', 'abs_pct_error'
]]

print("Top 5 Predictions with Largest Errors:")
print(top_errors)

In [ ]:
# Show predictions with smallest errors
top_accurate = results.nsmallest(5, 'abs_error')[[
    'forecast_date', 'actual', 'predicted', 'error', 'abs_pct_error'
]]

print("Top 5 Most Accurate Predictions:")
print(top_accurate)

## Conclusions

Based on the analysis:

1. **Directional Accuracy**: TimeGPT can predict price direction with varying accuracy
2. **Magnitude Predictions**: Confidence intervals provide useful uncertainty estimates
3. **Horizon Impact**: Shorter horizons generally perform better
4. **Market Conditions**: Performance varies across different market regimes

### Recommended Use Cases:

- Short-term trend analysis (1-7 days)
- Risk management using confidence intervals
- Complementary signal for trading strategies
- Volatility assessment

### Limitations:

- Past performance doesn't guarantee future results
- Bitcoin is influenced by many external factors
- Should be combined with other analysis methods